# CRIM Intervals:  Modules

### What You Can Do with this Notebook

* Learn how to do some basic filtering of one Dataframe to find the modules associated with homorhythmic passages



### A. Import Intervals and Other Code


In [33]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os


def convertTuple(tup):
    out = ""
    if isinstance(tup, tuple):
        out = ', '.join(tup)
    return out


MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)
else:
    print(MYDIR, "folder already exists.")
    
MUSDIR = ("Music_Files")
CHECK_FOLDER = os.path.isdir(MUSDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MUSDIR)
    print("created folder : ", MUSDIR)
else:
    print(MUSDIR, "folder already exists.")

saved_csv folder already exists.
Music_Files folder already exists.


## B. Importing a Piece

In [4]:
# Select a prefix:

# prefix = 'Music_Files/'
prefix = 'https://crimproject.org/mei/'

# Add your filename here
mei_file = 'CRIM_Model_0008.mei'

# and combine the strings and load the piece
url = prefix + mei_file
piece = importScore(url)

print(piece.metadata)

Previously imported piece detected.
{'title': 'Ave Maria', 'composer': 'Josquin Des Prés'}


In [5]:
# print(ImportedPiece.ngrams.__doc__)

### How can we find the ngrams associated with all the HR in a piece?

* We will first find the homorhythmic passages (where two or more voices have the same durations and same syllables).

* Then we will use the 'offsets' of those passages as a mask, returning only the harmonic ngrams associated with them.

* And then we can look for repeating harmonic ngrams in one or more pieces.



### Find HR, then Indices to List

find hr and then get the offset index as a list
hr has a 'multi'index, so we only need level 2, which is the offset

>`hr = piece.homorhythm()
hr_index = hr.index.get_level_values(2)`


In [7]:
hr = piece.homorhythm()
hr_index = hr.index.get_level_values(2)
hr
hr_index

Float64Index([ 244.0,  252.0,  256.0,  276.0,  302.0,  304.0,  316.0,  320.0,
               408.0,  604.0,  620.0,  622.0,  624.0,  644.0,  646.0,  736.0,
               748.0,  780.0,  796.0,  828.0,  840.0,  852.0,  952.0,  976.0,
              1040.0, 1072.0, 1184.0, 1200.0, 1208.0, 1224.0, 1236.0, 1240.0,
              1256.0, 1272.0],
             dtype='float64', name='Offset')

### Find all the harmonic ngrams in one piece (note interval and offset settings, and length)

>`all_har_ngs = piece.ngrams(df=piece.harmonic(kind="d", compound=False), n=2).fillna('')`

if you want the **modular ngrams** instead, us the following:

>`all_mods = piece.ngrams()`

In [9]:
all_har_ngs = piece.ngrams(df=piece.harmonic(kind="d", compound=False), n=2).fillna('')
all_har_ngs

,Bassus_Tenor,Bassus_Altus,Bassus_[Superius],Tenor_Altus,Tenor_[Superius],Altus_[Superius]
16.0,,,,,,"(5, 3)"
20.0,,,,,,"(3, 8)"
24.0,,,,,,"(8, 8)"
32.0,,,,"(5, 3)",,
36.0,,,,"(3, 1)",,
...,...,...,...,...,...,...
1244.0,"(3, 3)","(8, 5)","(5, 3)","(6, 3)","(3, 8)","(5, 6)"
1248.0,"(3, 5)","(5, 6)","(3, 3)","(3, 4)","(8, 6)","(6, 5)"
1252.0,,"(6, 8)",,"(4, 4)",,"(5, 3)"
1256.0,"(5, 8)","(8, 5)","(3, 8)","(4, 5)","(6, 8)","(3, 4)"


#### Now get a list of modules

>`all_ngs_list = all_har_ngs.index.to_list()`

And find the **intersection of both lists** (since sometimes the hr don't form modules, depending on surrounding rests, etc)

>`final_hr_list= hr_index.intersection(all_ngs_list)`



In [10]:
all_ngs_list = all_har_ngs.index.to_list()
final_hr_list= hr_index.intersection(all_ngs_list)
final_hr_list

Float64Index([ 244.0,  252.0,  256.0,  276.0,  302.0,  316.0,  320.0,  408.0,
               604.0,  620.0,  622.0,  624.0,  644.0,  646.0,  748.0,  780.0,
               796.0,  828.0,  840.0,  852.0,  952.0,  976.0, 1040.0, 1200.0,
              1208.0, 1236.0, 1240.0, 1256.0, 1272.0],
             dtype='float64', name='Offset')

### Now filter the Ngrams according to the index of HR passages

>`all_hr_ngrams = all_har_ngs.loc[final_hr_list]
all_hr_ngrams.fillna('')
all_hr_ngrams = all_hr_ngrams.fillna('').applymap(convertTuple)`

These are the ngrams of the hr passages

In [32]:

all_hr_ngrams = all_har_ngs.loc[final_hr_list]
all_hr_ngrams = all_hr_ngrams.fillna('').applymap(convertTuple)

### There are various ways to change the of NGrams 

You would need to use this with the hr mask above

- Diatonic, Chromatic, With-Quality, etc
- No Unisons
- Various lengths
- Exclude rests

In [19]:
mel = piece.melodic(kind="c")
har = piece.harmonic(kind="c")
ngrams = piece.ngrams(df=har, other=mel, n=3).fillna('')
ngrams.head()

,Bassus_Tenor,Bassus_Altus,Bassus_[Superius],Tenor_Altus,Tenor_[Superius],Altus_[Superius]
16.0,,,,,,"19_5, 16_Held, 12"
20.0,,,,,,"16_Held, 12_0, 12"
32.0,,,,"7_5, 4_Held, 0",,
36.0,,,,"4_Held, 0_0, 0",,
48.0,"19_5, 16_Held, 12",,,,,


In [20]:
nr_no_unisons = piece.notes(combineUnisons=True)
mel = piece.melodic(df=nr_no_unisons, kind="d")
har = piece.harmonic(kind="d", compound=False)
ngrams = piece.ngrams(df=har, other=mel).fillna('')
ngrams.head()

,Bassus_Tenor,Bassus_Altus,Bassus_[Superius],Tenor_Altus,Tenor_[Superius],Altus_[Superius]
16.0,,,,,,"5_4, 3_Held, 8"
20.0,,,,,,"3_Held, 8_Held, 8"
32.0,,,,"5_4, 3_Held, 1",,
36.0,,,,"3_Held, 1_Held, 1",,
48.0,"5_4, 3_Held, 8",,,,,


## Search the Ngrams of the HR 

Now you can also search within those results for anything you like

In [34]:

@interact
def har_ngram_search(my_search="", df = fixed(all_hr_ngrams)):
    
    df2 = all_hr_ngrams.copy()
    filtered_ngrams = df2[df2.apply(lambda x: x.astype(str).str.contains(my_search).any(), axis=1)].copy()
    
    pd.set_option('max_columns', None)
    return filtered_ngrams.fillna("-").reset_index().applymap(str).style.applymap(lambda x: "background: #ccebc4" if re.search(my_search, x) else "")

interactive(children=(Text(value='', description='my_search'), Output()), _dom_classes=('widget-interact',))